In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

c:\Users\jiahe\miniconda3\envs\autogluon\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load your level-1 (OOF) training data
oof_rf = pd.read_csv(r"C:\Users\jiahe\OneDrive\Desktop\kaggle\oof_lgb_rf.csv")
oof_tabm = pd.read_csv(r"C:\Users\jiahe\OneDrive\Desktop\kaggle\oof_tabm.csv")

# Load your level-1 (test) data
test_rf = pd.read_csv(r"C:\Users\jiahe\OneDrive\Desktop\kaggle\test_lgb_rf.csv")
test_tabm = pd.read_csv(r"C:\Users\jiahe\OneDrive\Desktop\kaggle\test_tabm.csv")

In [3]:
train_stack = (
    oof_rf.merge(oof_tabm[["id", "oof_tabm"]], on="id", how="inner")
    .rename(columns={
        "oof_lgb": "pred_lgb",
        "oof_rf": "pred_rf",
        "oof_blend": "pred_blend",
        "oof_tabm": "pred_tabm",
        "y_true": "accident_risk"
    })
)
print(train_stack.head())

   id  pred_lgb   pred_rf  pred_blend  accident_risk  pred_tabm
0   0  0.128810  0.128461    0.128724           0.13   0.130713
1   1  0.325818  0.325341    0.325701           0.35   0.330035
2   2  0.387836  0.388745    0.388060           0.30   0.385851
3   3  0.126874  0.129990    0.127642           0.21   0.134695
4   4  0.471175  0.467399    0.470245           0.56   0.471517


In [4]:
test_stack = (
    test_rf.merge(test_tabm[["id", "test_tabm"]], on="id", how="inner")
    .rename(columns={
        "test_lgb": "pred_lgb",
        "test_rf": "pred_rf",
        "test_blend": "pred_blend",
        "test_tabm": "pred_tabm"
    })
)
print(test_stack.head())

       id  pred_lgb   pred_rf  pred_blend  pred_tabm
0  517754  0.294137  0.299005    0.295361   0.294441
1  517755  0.119736  0.119776    0.119731   0.119518
2  517756  0.185015  0.181694    0.184177   0.185151
3  517757  0.308525  0.313961    0.310042   0.310965
4  517758  0.394309  0.404265    0.396693   0.395622


In [5]:
label = "accident_risk"
train_ag = TabularDataset(train_stack)
test_ag = TabularDataset(test_stack)

predictor = TabularPredictor(
    label=label,
    problem_type="regression",
    eval_metric="rmse"
).fit(
    train_data=train_ag,
    presets="best_quality",
    time_limit=1800,  # 30 mins; adjust as you like
    verbosity=1
)

No path specified. Models will be saved in: "AutogluonModels\ag-20251029_013750"
2025-10-28 21:37:52,857	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-10-28 21:37:57,001	INFO worker.py:1852 -- Started a local Ray instance.
(_dystack pid=22320) 	Warning: Reducing model 'n_estimators' from 300 -> 110 due to low time. Expected time usage reduced from 378.5s -> 139.6s...
(_dystack pid=22320) 	Warning: Reducing model 'n_estimators' from 300 -> 66 due to low time. Expected time usage reduced from 46.8s -> 10.4s...
(_dystack pid=22320) 	Not enough time to generate out-of-fold predictions for model. Estimated time required was 32.67s compared to 10s of available time.
(_dystack pid=22320) 	Warning: Exception caused NeuralNetFastAI_BAG_L1 to fail during training (ImportError)... Skipping this model.
(_dystack pid=22320) 		ray::_ray_fit() (pid=8284, ip=127.0.0.1)
(_dystack pid=22320) ModuleNo

In [6]:
test_pred = predictor.predict(test_ag)
print(test_pred.describe())

# Prepare final submission
submission = pd.DataFrame({
    "id": test_stack["id"],
    "accident_risk": test_pred
})
submission.to_csv("submission_autogluon.csv", index=False)
print("✅ Saved submission_autogluon.csv")

count    172585.000000
mean          0.352397
std           0.156408
min           0.033629
25%           0.242621
50%           0.336175
75%           0.455224
max           0.854276
Name: accident_risk, dtype: float64
✅ Saved submission_autogluon.csv
